In [8]:
k = 0
for i in range(0, 8):
    for j in range(0, 8):
        print("{0:06b}".format(k), end=" ")
        k += 1
    print("")
print("")

000000 000001 000010 000011 000100 000101 000110 000111 
001000 001001 001010 001011 001100 001101 001110 001111 
010000 010001 010010 010011 010100 010101 010110 010111 
011000 011001 011010 011011 011100 011101 011110 011111 
100000 100001 100010 100011 100100 100101 100110 100111 
101000 101001 101010 101011 101100 101101 101110 101111 
110000 110001 110010 110011 110100 110101 110110 110111 
111000 111001 111010 111011 111100 111101 111110 111111 



In [30]:
import math
a = 4 # Starts from 0
i = 0 # Starts from 0
p = 8
pi = (a + (-1)**(a % 2)*(-1)**(i % 2)*2**math.floor(i/2)) % p
pialt = (a + -1**(a+i)*2**math.floor(i/2)) % p
print(pi)
print(pialt)

5
3


In [49]:
import math
import copy
import random
import numpy as np
def get_node_at_distance_in_direction(n, d, dir, rows, cols):
    (r, c) = n // rows, n % cols
    if dir == 'E':
        dest_coord = (r, (c+d) % cols)
    elif dir == 'S':
        dest_coord = ((r+d) % cols, c)
    elif dir == 'W':
        dest_coord = (r, (c-d) % cols)
    else: # 'N'
        dest_coord = ((r-d) % cols, c)
    r_d, c_d = dest_coord
    return (r_d)*cols + c_d

def compute_expected_data_allreduce(data, p):
    expected_res = np.zeros(p)
    # Compute expected result (Cols = hosts, rows = same data on different hosts)
    for i in range(0, p):
        for j in range(0, p):
            expected_res[i] += data[j][i]
    return expected_res

# Algo:
# 1. I take the tuple representing the coordinates
# 2. I proceed one dimension at a time
#    3. If I am an even node wrt that coordinate, I first send in positive direction, then in negative (otherwise the other way around)

dimensions = [16, 16]
#dimensions = [16]
p = 1
for d in dimensions:
    p *= d

data = np.zeros((p, p))
sum_of_distances = 0
sum_of_distances_ref = 0
bw_new = 0
bw_ref = 0
bw_ideal = 0
directions_even_even = ['E', 'S', 'W', 'N']
directions_even_odd = ['W', 'S', 'E', 'N']
directions_odd_even = ['E', 'N', 'W', 'S']
directions_odd_odd = ['W', 'N', 'E', 'S']
# Initialize random starting data
for i in range(0, p):
    for j in range(0, p):
        data[i][j] = random.randint(0, 100)

expected_res = compute_expected_data_allreduce(data, p)

# 1D version
if len(dimensions) == 1:
    for i in range(0, int(math.log2(p))):
        distances = [1, 1, 3, 5, 11, 21, 43] # Jacobsthal sequence!! (https://oeis.org/search?q=1%2C+1%2C+3%2C+5%2C+11%2C+21&language=english&go=Search)
        data_c = copy.deepcopy(data)
        # Bit array to check that no one sends the same data to more than one node
        recv_from = np.zeros(p)
        for j in range(0, p):       
            distance = distances[i]
            if j % 2 == 0 and i % 2 == 0: # Even id, even iteration (starting form 0)
                peer = (j+distance) % p
            elif j % 2 == 0 and i % 2 != 0: # Even id, odd iteration
                peer = (j-distance) % p
            elif j % 2 != 0 and i % 2 == 0: # Odd id, even iteration
                peer = (j-distance) % p
            else: # Odd id, odd iteration
                peer = (j+distance) % p
            # Check that each node sends at most at one node
            if recv_from[peer]:
                print("Double recv")
                exit(-1)
            print("Step " + str(i) + ": " + str(j) + "->" + str(peer))
            recv_from[peer] = 1
            # Aggregate
            for k in range(0, p):
                data_c[j][k] += data[peer][k]
        # Check that everyone sent something
        for k in range(0, p):
            if not recv_from[k]:
                print("Nosent")
                exit(-1)

        # Compute some stats
        sum_of_distances += distance 
        sum_of_distances_ref += 2**i
        bw_ideal += (1/2**(i+1)) 
        bw_new += (1/2**(i+1))*distance
        bw_ref += (1/2**(i+1))*2**i
        data = copy.deepcopy(data_c)
# 2D version
elif len(dimensions) == 2:
    rows = dimensions[0]
    cols = dimensions[1]
    for i in range(0, int(math.log2(p))):
        distances = [1, 1, 1, 1, 3, 3, 5, 5, 11, 11, 21, 21, 43, 43]
        data_c = copy.deepcopy(data)
        # Bit array to check that no one sends the same data to more than one node
        recv_from = np.zeros(p)
        for j in range(0, p):       
            distance = distances[i]
            row = j // rows
            col = j % cols
            if row % 2 == 0 and col % 2 == 0: # Even row, even col
                peer = get_node_at_distance_in_direction(j, distances[i], directions_even_even[i % 4], rows, cols)        
            elif row % 2 == 0 and col % 2 != 0: # Even row, odd col
                peer = get_node_at_distance_in_direction(j, distances[i], directions_even_odd[i % 4], rows, cols)
            elif row % 2 != 0 and col % 2 == 0: # Odd row, even col
                peer = get_node_at_distance_in_direction(j, distances[i], directions_odd_even[i % 4], rows, cols)
            else: # Odd row, odd col
                peer = get_node_at_distance_in_direction(j, distances[i], directions_odd_odd[i % 4], rows, cols)

            # Check that each node sends at most at one node
            if recv_from[peer]:
                print("Double recv")
                exit(-1)
            recv_from[peer] = 1
            # Aggregate
            for k in range(0, p):
                data_c[j][k] += data[peer][k]
        # Check that everyone sent something
        for k in range(0, p):
            if not recv_from[k]:
                print("Nosent")
                exit(-1)

        # Compute some stats
        sum_of_distances += distance 
        sum_of_distances_ref += 2**math.floor(i/2)
        bw_ideal += (1/2**(i+1)) 
        bw_new += (1/2**(i+1))*distance
        bw_ref += (1/2**(i+1))*2**math.floor(i/2)
        data = copy.deepcopy(data_c)
else:
    pass

# Check correctness of the result
for i in range(0, p):
    for j in range(0, p):
        if data[i][j] != expected_res[j]:
            print("FAIL!")
            break        
#print(data)
#print(expected_res)
print("Sum of distances: " + str(sum_of_distances) + " ref: " + str(sum_of_distances_ref))
print("Bw new: " + str(bw_new) + " bw ref: " + str(bw_ref) + " bw ideal: " + str(bw_ideal))


Step 0: 0->1
Step 0: 1->0
Step 0: 2->3
Step 0: 3->2
Step 0: 4->5
Step 0: 5->4
Step 0: 6->7
Step 0: 7->6
Step 0: 8->9
Step 0: 9->8
Step 0: 10->11
Step 0: 11->10
Step 0: 12->13
Step 0: 13->12
Double recv
Step 0: 14->0
Nosent
Step 1: 0->14
Step 1: 1->2
Step 1: 2->1
Step 1: 3->4
Step 1: 4->3
Step 1: 5->6
Step 1: 6->5
Step 1: 7->8
Step 1: 8->7
Step 1: 9->10
Step 1: 10->9
Step 1: 11->12
Step 1: 12->11
Double recv
Step 1: 13->14
Step 1: 14->13
Nosent
Step 2: 0->3
Step 2: 1->13
Step 2: 2->5
Step 2: 3->0
Step 2: 4->7
Step 2: 5->2
Step 2: 6->9
Step 2: 7->4
Step 2: 8->11
Step 2: 9->6
Double recv
Step 2: 10->13
Step 2: 11->8
Double recv
Step 2: 12->0
Step 2: 13->10
Double recv
Step 2: 14->2
Nosent
Nosent
Nosent
FAIL!
FAIL!
FAIL!
FAIL!
FAIL!
FAIL!
FAIL!
FAIL!
FAIL!
FAIL!
FAIL!
FAIL!
FAIL!
FAIL!
FAIL!
Sum of distances: 5 ref: 7
Bw new: 1.125 bw ref: 1.5 bw ideal: 0.875
